In [1]:
import numpy as np
import tensorflow as tf
import os

import librosa
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

opj = os.path.join

In [2]:
gtzan_path = r'datasets\gtzan'
genres_dir = opj(gtzan_path, 'genres_original')

In [3]:
def load_data_by_classes(dataset_path):
    data = []
    labels = []
    classes = os.listdir(dataset_path)
    for cl in tqdm(classes):
        full_p = opj(dataset_path, cl)
        for file in os.listdir(full_p):
            for i in range(0, 20, 2):
                filepath = os.path.join(full_p, file)  
                try:
                    wavedata, _ = librosa.load(filepath, sr=None, mono=True, offset=i, duration=2)
                except:
                    continue
                wavedata = wavedata[:, np.newaxis]
                data.append(wavedata)
                labels.append(cl)

    return data, labels

In [4]:
data, labels = load_data_by_classes(genres_dir)

 50%|█████████████████████████████████████████▌                                         | 5/10 [00:01<00:01,  2.71it/s]C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.89it/s]


In [ ]:
sizes = [len(item) for item in data]
min_size = min(sizes)

data = [item[:min_size] for item in data]

# int(min_size/16)

In [ ]:
data = np.array(data)
# data = data[..., np.newaxis]


le = LabelEncoder()
labels = le.fit_transform(labels)
labels_1hot = to_categorical(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.15)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras.layers import Conv1D, Dropout, LSTM, TimeDistributed, Activation, Dense, Input, MaxPooling1D, Lambda, Flatten, BatchNormalization, LSTM

In [ ]:
N_LAYERS = 4
CONV_FILTER_COUNT = 64
FILTER_LENGTH = 25

GENRES = 10

In [ ]:
import tensorflow.keras.backend as K

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import RMSprop

In [ ]:
LSTM_COUNT = 256
BATCH_SIZE = 32
EPOCH_COUNT = 80

kernel_size = 100

In [ ]:
model = Sequential()

model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=data.shape[1:]))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=10, activation="softmax"))

model.compile(loss='categorical_crossentropy' , optimizer=RMSprop(learning_rate=0.0001) , metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 24, batch_size=4)